# Yelp API - Lab


## Introduction 

Now that we've seen how the Yelp API works and some basic Folium visualizations, it's time to put those skills to work in order to create a working map! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to: 
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and perform data analysis on the data returned
* Perform pagination to retrieve troves of data!
* Create a simple geographical system on to view information about selected businesses, at a given location. 

## Problem Introduction

You've now worked with some API calls, but we have yet to see how to retrieve a more complete dataset in a programmatic manner. Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retrieving 50 at a time. Processes such as these are often referred to as pagination.

In this lab, you will define a search and then paginate over the results to retrieve all of the results. You'll then parse these responses as a DataFrame (for further exploration) and create a map using Folium to visualize the results geographically.

## Part I - Make the Initial Request

Start by making an initial request to the Yelp API. Your search must include at least 2 parameters: **term** and **location**. For example, you might search for pizza restaurants in NYC. The term and location is up to you, but make the request below.

In [5]:
client_id = "WksaDnsw4SSoKEBqkyUwkQ" #Your client ID goes here (as a string)
api_key = "Ie1oWbJxzj8IAn348fY_pqYlPmjgwWNcBtP793ZsHmZOfRgohOHfHf0z2JKunKwdgTU90hEYpQF32KqY3y2ck_eqHJ_TEU8YqmJwgHAWMhkpL1tcI6xuNRKusObZXHYx" #Your api key goes here (as a string)

In [6]:
import requests

In [7]:
term = 'salata'
location = 'New York NY'

url = 'https://api.yelp.com/v3/businesses/search'

headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }

url_params = {
                'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
            }
response = requests.get(url, headers=headers, params=url_params)
print(response)
print(type(response.text))
print(response.text[:1000])

<Response [200]>
<class 'str'>
{"businesses": [{"id": "l4fwf8VwAokv3MWT45Ab7g", "alias": "bunna-cafe-brooklyn", "name": "Bunna Cafe", "image_url": "https://s3-media4.fl.yelpcdn.com/bphoto/jGKIcCZ2lx23kKX2YRug7g/o.jpg", "is_closed": false, "url": "https://www.yelp.com/biz/bunna-cafe-brooklyn?adjust_creative=WksaDnsw4SSoKEBqkyUwkQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=WksaDnsw4SSoKEBqkyUwkQ", "review_count": 528, "categories": [{"alias": "ethiopian", "title": "Ethiopian"}, {"alias": "vegan", "title": "Vegan"}, {"alias": "african", "title": "African"}], "rating": 4.5, "coordinates": {"latitude": 40.705042, "longitude": -73.928078}, "transactions": ["delivery", "pickup"], "price": "$$", "location": {"address1": "1084 Flushing Ave", "address2": "", "address3": "", "city": "Brooklyn", "zip_code": "11237", "country": "US", "state": "NY", "display_address": ["1084 Flushing Ave", "Brooklyn, NY 11237"]}, "phone": "+13472952227", "display_phone": "(347) 295-2227", 

## Pagination

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. Your final goal will be to reformat the data concerning the businesses themselves into a pandas DataFrame from the json objects.

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and are also can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

In [8]:
response.json().keys()

dict_keys(['businesses', 'total', 'region'])

In [9]:
response.json()['total']

2000

In [10]:
businesses = dict(response.json())

In [11]:
len(businesses)

3

In [12]:
businesses.keys()

dict_keys(['businesses', 'total', 'region'])

In [13]:
len(businesses['businesses'])

20

In [14]:
import time

In [15]:
def query_yelp_businesses(search_term, search_location, api_key, offset=0, time_to_wait=5):
    term = search_term
    location = search_location

    url = 'https://api.yelp.com/v3/businesses/search'

    headers = {
            'Authorization': 'Bearer {}'.format(api_key),
        }

    url_params = {
                    'term': term.replace(' ', '+'),
                    'location': location.replace(' ', '+'),
                    'offset': offset
                }
    response = requests.get(url, headers=headers, params=url_params)
    time.sleep(time_to_wait)
    
    return response

In [16]:
response = query_yelp_businesses("salata", "Omaha NE", api_key, offset=0)
dict(response.json())['total']

74

In [ ]:
response = query_yelp_businesses("salata", "Omaha NE", api_key, offset=0)
# Get the total number of responses
total = dict(response.json())['total']

offset = 0
responses = []
while offset < total:
    responses.append(query_yelp_businesses("salata", "Omaha NE", api_key, offset=offset))
    offset += 50

In [ ]:
len(responses)

In [ ]:
for response in responses:
    print(dict(response.json())['businesses'][0].keys())

In [ ]:
for response in responses:
    json_dict = dict(response.json())
    business_list = json_dict['businesses']
    for business in business_list:
        print(f"{business['name']} {business['coordinates']} {business['location']}")

In [ ]:
import pandas as pd

In [ ]:
dfs = []
for response in responses:
    json_dict = dict(response.json())['businesses']
    dfs.append(pd.DataFrame.from_dict(json_dict))

In [ ]:
df = pd.concat(dfs)

In [ ]:
df.reset_index(inplace=True)

In [ ]:
df.head()

In [ ]:
df.tail()

## Exploratory Analysis

Take the restaurants from the previous question and do an initial exploratory analysis. At minimum, this should include looking at the distribution of features such as price, rating and number of reviews as well as the relations between these dimensions.

In [ ]:
import matplotlib
%matplotlib inline

In [ ]:
df['price'].dropna().apply(lambda x: len(x)).value_counts().plot(kind='bar')

In [ ]:
df['rating'].dropna().value_counts().plot(kind='bar')

In [ ]:
df.columns

In [ ]:
df['review_count'].dropna().plot(kind='hist')

In [ ]:
df['price'] = df['price'].fillna(value='$').apply(lambda x: len(x))

In [ ]:
df_to_analyze = df.loc[:, ['review_count', 'rating', 'price']].fillna(value=0.0)

In [ ]:
pd.plotting.scatter_matrix(df_to_analyze)

In [ ]:
df_to_analyze.corr()

## Mapping

Look at the initial Yelp example and try and make a map using Folium of the restaurants you retrieved. Be sure to also add popups to the markers giving some basic information such as name, rating and price.

In [ ]:
import folium
import numpy as np

In [2]:
import ast
ast.literal_eval(df['location'][0])

NameError: name 'df' is not defined

In [4]:
type(ast.literal_eval("{'test':0}"))

dict

In [ ]:
lat = 
long = -0.14

#Create a map of the area
base_map = folium.Map([lat, long], zoom_start=13)

points = list(zip(x, y))
for p in points:
    lat = p[0]
    long = p[1]
    marker = folium.Marker(location=[lat, long])
    marker.add_to(base_map)

for p in points:
    lat = p[0]
    long = p[1]
    popup_text = "Latitude: {}, Longitude: {}".format(lat,long)
    popup = folium.Popup(popup_text, parse_html=True)
    marker = folium.Marker(location=[lat, long], popup=popup)
    marker.add_to(base_map)
base_map

## Summary

Nice work! In this lab, you synthesized your skills for the day, making multiple API calls to Yelp in order to paginate through a results set, performing some basic exploratory analysis and then creating a nice map visual to display the results! Well done!